In [6]:
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
import time

def text_overwrite_to_image(img, text, position):
    """テキストを画像に描画する関数"""

    # 画像の横サイズを取得
    img_width = img.shape[1]

    # 黒四角形（拡張領域）を作成
    extension_height = 40
    black_rectangle = np.zeros((extension_height, img_width, 3), dtype=np.uint8)

    # 元画像と黒四角形結合
    extended_img = np.vstack((black_rectangle, img))

    # 文字の設定
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)
    font_thickness = 2

    # 画像にテキストを上書き
    cv2.putText(extended_img,
                text=text,
                org=position,
                fontFace=font,
                fontScale=font_scale,
                color=font_color,
                thickness=font_thickness)

    return extended_img

def detect_faces(frame, person_box, face_cascade, smile_cascade):
    """顔を検出する関数"""

    # 物体検出領域から顔を検出
    x1, y1, x2, y2 = int(person_box[0]), int(person_box[1]), int(person_box[2]), int(person_box[3])
    roi_gray = cv2.cvtColor(frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(roi_gray, 1.1, 4)

    # 顔検出結果をバウンディングボックスで描画
    is_smile = False
    for (fx, fy, fw, fh) in faces:
        roi_face_gray = roi_gray[fy:fy+fh, fx:fx+fw]
        face_top_left = (x1 + fx, y1 + fy)
        face_bottom_right = (x1 + fx + fw, y1 + fy + fh)
        cv2.rectangle(frame, face_top_left, face_bottom_right, (0, 255, 0), 2)

        # 笑顔検出
        smiles = smile_cascade.detectMultiScale(roi_face_gray, 1.8, 20)
        if len(smiles) > 0:
            is_smile = True
        for (sx, sy, sw, sh) in smiles:
            smile_top_left = (x1 + fx + sx, y1 + fy + sy)
            smile_bottom_right = (x1 + fx + sx + sw, y1 + fy + sy + sh)
            cv2.rectangle(frame, smile_top_left, smile_bottom_right, (255, 0, 0), 2)

    return frame, is_smile
    
# モデルを設定
model = YOLO('yolov8n.pt')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# 動画撮影開始
cap = cv2.VideoCapture(0)

# 画像保存
file_counter = 0
last_saved_time = 0
folder = Path('img-smile')
folder.mkdir(exist_ok=True)

while cap.isOpened():
    # フレームを抽出する
    ret, frame = cap.read()

    if ret:
        # 物体検出
        results = model(frame, verbose=False, classes=0)
        img_annotated = results[0].plot()

        # 分析
        class_list = results[0].boxes.cls
        count_class = len(class_list)

        # 物体検出部分の領域を取得
        person_boxes = results[0].boxes.xyxy

        # 顔検出
        count_smile = 0
        for box in person_boxes:
            img_annotated, is_smile = detect_faces(img_annotated, box, face_cascade, smile_cascade)
            if is_smile:
                count_smile += 1
        
        # 画像を表示
        img_annotated = text_overwrite_to_image(img_annotated, f'person={count_class}', (10, 25))
        img_annotated = text_overwrite_to_image(img_annotated, f'smile={count_smile}',(10, 25))
        cv2.imshow('Playing', img_annotated)

        # 人数と笑顔数の一致で画像を保存
        current_time = time.time()
        if count_class == count_smile and (current_time - last_saved_time) > 10:
            filename = folder / f'image_{file_counter:04d}.jpg'
            cv2.imwrite(filename, frame)
            file_counter += 1
            last_saved_time = current_time

        # q キーが押されたらウィンドウを閉じる
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# 解放処理
cap.release()
cv2.destroyAllWindows()